In [322]:
def parse(text):
    lines = text.split("\n")

    return [list(map(lambda x: int(x), line.split(" "))) for line in lines]

In [333]:
import re 

def get_starting_point(map: list[list[str]]) -> (int, int):
    for y in range(len(map)):
        for x in range(len(map[y])):
            if map[y][x] == "S":
                return (x, y)


def get_starting_point_nodes(map: list[list[str]]) -> (int, int):
    start = get_starting_point(map)
    starting_nodes = []

    for y in range(max(0, start[1] - 1), min(len(map), start[1] + 2)):
        for x in range(max(0, start[0] - 1), min(len(map[y]), start[0] + 2)):
            if start in get_next_nodes(map, (x, y)):
                starting_nodes.append((x, y))

    return starting_nodes


def get_next_nodes(map: list[list[str]], currentPoint: tuple) -> list[tuple]:
    currentX, currentY = currentPoint
    candidates = []

    if map[currentY][currentX] == "|":
        candidates = [(currentX, currentY + 1), (currentX, currentY - 1)]

    if map[currentY][currentX] == "-":
        candidates = [(currentX + 1, currentY), (currentX - 1, currentY)]

    if map[currentY][currentX] == "L":
        candidates = [(currentX, currentY - 1), (currentX + 1, currentY)]

    if map[currentY][currentX] == "J":
        candidates = [(currentX, currentY - 1), (currentX - 1, currentY)]

    if map[currentY][currentX] == "7":
        candidates = [(currentX, currentY + 1), (currentX - 1, currentY)]

    if map[currentY][currentX] == "F":
        candidates = [(currentX, currentY + 1), (currentX + 1, currentY)]

    nextNodes = []
    for nextX, nextY in candidates:
        if nextX < len(map[0]) and nextX >= 0 and nextY < len(map) and nextY >= 0:
            nextNodes.append((nextX, nextY))

    return nextNodes


def splice_string(string: str, index: int, replacement: str) -> str:
    return string[:index] + replacement + string[index + 1:]

def get_map_stats(map: list[list[str]]) -> dict:
    maximum = 1
    nodes = get_starting_point_nodes(map)
    computed_nodes = [get_starting_point(map)]
    done = False
    enclosed_count = 0

    while done == False:
        maximum += 1
        for i, node in enumerate(nodes):
            computed_nodes.append(node)
            candidates = get_next_nodes(map, node)

            next_node = None

            for candidate in candidates:
                if candidate not in computed_nodes:
                    next_node = candidate

            if next_node:
                if next_node in nodes:
                    computed_nodes.append(next_node)
                    done = True

                nodes[i] = next_node
            else:
                done = True

    for y in range(len(map)):
        for x in range(len(map[y])):
            if (x, y) not in computed_nodes:
                map[y] = splice_string(map[y], x, ".")

    enclosed_count = 0
    for line in map:
        line = re.sub(r"L-*7|F-*J", "|", line)

        vertical_count = 0
        for char in line:
            if char == "|":
                vertical_count += 1

            if char == "." and vertical_count % 2 == 1:
                enclosed_count += 1

    return (maximum, enclosed_count)

In [336]:
test_input = """..F7.
.FJ|.
SJ.L7
|F--J
LJ..."""

test_map = test_input.split("\n")

assert get_map_stats(test_map)[0] == 8

In [337]:
import re

test_input = """...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
..........."""

test_map = test_input.split("\n")

assert get_map_stats(test_map)[1] == 4

In [330]:
map = open("10.txt").read().split("\n")

maximum, enclosed_count = get_map_stats(map)

print("Part 1: ", maximum)
print("Part 2: ", enclosed_count)

Part 1:  6806
...........................................................................................F7...............................................
...........................................................................................||...............................................
...........................................................................................||F--7F-7........................................
......................................................................................F-7..|||F-J|FJ........................................
...................................................................................F7.|FJF7||||..||F7.......................................
...........................................................................F-7..F7FJL7||.|LJ|||.FJLJ|.......................................
......................................................................F7...L7|F-J|L-7LJL7L7FJ|L7|F--J.......................................